In [1]:
import numpy as np
import tensorflow as tf
import pickle

In [2]:
def scale_img(image):
    return_img = np.zeros_like(image, dtype=float)

    r_img = image[:, 0:1024]
    r_avg = np.average(r_img)
    r_std = np.std(r_img)
    r_img = (r_img - r_avg) / r_std
    return_img[:, 0:1024] = r_img

    g_img = image[:, 1024:2048]
    g_avg = np.average(g_img)
    g_std = np.std(g_img)
    g_img = (g_img - g_avg) / g_std
    return_img[:, 1024:2048] = g_img

    b_img = image[:, 2048:]
    b_avg = np.average(b_img)
    b_std = np.std(b_img)
    b_img = (b_img - b_avg) / b_std
    return_img[:, 2048:] = b_img

    return return_img


def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict


def unpickle_all(path):
    train_images = np.zeros((50000, 32, 32, 3))
    train_stack = np.zeros((50000, 3072))
    test_images = np.zeros((10000, 32, 32, 3))
    train_labels = np.zeros((50000))
    test_labels = np.zeros((10000))
    for i in range(1, 6):
        j = i - 1
        cifar10 = unpickle(path + '/data_batch_{}'.format(i))
        train_labels[j * 10000:(j + 1) * 10000] = cifar10['labels']
        train_stack[j * 10000:(j + 1) * 10000, :] = cifar10['data']

    train_stack = scale_img(train_stack)
    train_images = np.swapaxes(np.reshape(train_stack, (50000, 32, 32, 3), order='F'), axis1=1, axis2=2)

    cifar10 = unpickle(path + '/test_batch')
    test_labels = cifar10['labels']
    data = scale_img(cifar10['data'])
    test_images = np.swapaxes(np.reshape(data, (10000, 32, 32, 3), order='F'), axis1=1, axis2=2)

    return train_images, train_labels, test_images, test_labels

In [3]:
images, labels, test_images, test_labels = unpickle_all('./storage/CIFAR-10')

In [4]:
tf.reset_default_graph()
batch_size = 100
total_batch = int(images.shape[0] / batch_size)

In [8]:
x = tf.placeholder('float32', [None, 32, 32, 3], name="X")
y = tf.placeholder('int64', [None, ], name="Y")

# OUT_CL_1 : n x 14 x 14 x 6
# OUT_CL_2 : n x 5 x 5 x 16 = n x 400
# OUT_DENSE_1 : n x 120
# OUT_DENSE_2 : n x 84

W_CL_1 = tf.Variable(np.zeros((5, 5, 3, 6)), dtype="float32", name="W_CL_1")
b_CL_1 = tf.Variable(np.zeros((1, 1, 1, 6)), dtype="float32", name="b_CL_1")
W_CL_2 = tf.Variable(np.zeros((5, 5, 6, 16)), dtype="float32", name="W_CL_2")
b_CL_2 = tf.Variable(np.zeros((1, 1, 1, 16)), dtype="float32", name="b_CL_2")

W_DENSE_1 = tf.Variable(np.zeros((400, 120)), dtype="float32", name="W_DENSE_1")
b_DENSE_1 = tf.Variable(np.zeros((1, 120)), dtype="float32", name="b_DENSE_1")
W_DENSE_2 = tf.Variable(np.zeros((120, 84)), dtype="float32", name="W_DENSE_2")
b_DENSE_2 = tf.Variable(np.zeros((1, 84)), dtype="float32", name="b_DENSE_2")
W_LOGITS_1 = tf.Variable(np.zeros((84, 10)), dtype="float32", name="W_LOGITS_1")
b_LOGITS_1 = tf.Variable(np.zeros((1, 10)), dtype="float32", name="b_LOGITS_1")


with tf.name_scope("CL1"):
    CL_1 = tf.nn.conv2d(x, W_CL_1, strides=[1, 1, 1, 1], padding="VALID", name="CL_1")
    RELU_CL_1 = tf.nn.relu(tf.add(CL_1, b_CL_1), name="RELU_CL_1")
    MAXPOOL_1 = tf.nn.max_pool(RELU_CL_1, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding="VALID", name="MAXPOOL_1")

with tf.name_scope("CL2"):
    CL_2 = tf.nn.conv2d(MAXPOOL_1, W_CL_2, strides=[1, 1, 1, 1], padding="VALID", name="CL_2")
    RELU_CL_2 = tf.nn.relu(tf.add(CL_2, b_CL_2), name="RELU_CL_2")
    MAXPOOL_2 = tf.nn.max_pool(RELU_CL_2, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding="VALID", name="MAXPOOL_2")
    FLAT_2 = tf.layers.flatten(MAXPOOL_2, name="FLAT_2")

with tf.name_scope("DENSE_1"):
    DENSE_1 = tf.add(tf.matmul(FLAT_2, W_DENSE_1), b_DENSE_1)
    RELU_DENSE_1 = tf.nn.relu(DENSE_1, name="RELU_DENSE_1")

with tf.name_scope("DENSE_2"):
    DENSE_2 = tf.add(tf.matmul(RELU_DENSE_1, W_DENSE_2), b_DENSE_2)
    RELU_DENSE_2 = tf.nn.relu(DENSE_2, name="RELU_DENSE_2")

with tf.name_scope("LOGITS_1"):
    LOGITS_1 = tf.add(tf.matmul(RELU_DENSE_2, W_LOGITS_1), b_LOGITS_1)

with tf.name_scope("PREDICTIONS"):
    y_hat = tf.nn.softmax(LOGITS_1, name="y_hat")
    predictions = tf.argmax(y_hat, 1)

with tf.name_scope("ACCURACY"):
    compare = tf.equal(predictions, y)
    accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))

saver = tf.train.Saver()

In [24]:
with tf.Session() as sess:

    saver.restore(sess, "./data/model/model.ckpt")
    summary_writer = tf.summary.FileWriter('data/logs')
    summary_writer.add_graph(sess.graph)
    avg_train_accuracy = []
    total_batch = int(images.shape[0] / batch_size)
    '''
    for i in range(2):
        batch_xs = images[i * batch_size:(i + 1) * batch_size, :, :, :]
        batch_ys = labels[i * batch_size:(i + 1) * batch_size]
        train_accuracy = sess.run([accuracy],feed_dict={x: batch_xs, y: batch_ys})
        avg_train_accuracy.append(train_accuracy)
   
    test_accuracy = sess.run([accuracy], feed_dict={x: test_images, y: test_labels})
    print("Train accuracy= {}  Test accuracy= {}".format(avg_train_accuracy, test_accuracy[0]))
    '''
    
    pred = sess.run([predictions], feed_dict={x: images[355:385,:,:,:]})
    pred = np.asarray(pred)
    pred = pred[0]
    truth = np.asarray(labels[355:385]).astype(int)
    
    print("Predictions : {}".format(pred)) 
    print("Ground truth: {}".format(truth))

INFO:tensorflow:Restoring parameters from ./data/model/model.ckpt
Predictions : [6 3 9 3 6 9 6 7 4 1 8 7 3 6 9 3 0 4 0 5 1 0 3 4 8 5 6 7 2 3]
Ground truth: [6 2 9 2 5 9 6 7 4 1 8 7 3 6 9 3 0 4 0 5 1 0 3 4 8 5 4 7 2 3]
